In [28]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from datetime import datetime

# Load datasets with explicit date parsing
stock_df = pd.read_csv('data/AMZN_historical_data.csv')
news_df = pd.read_csv('data/raw_analyst_ratings_1.csv')
# Set datetime index for stock data


# Display basic information and the first few rows of each dataset
stock_df_info = stock_df.info()
news_df_info = news_df.info()

# Step 1: Date Alignment


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6846 entries, 0 to 6845
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          6846 non-null   object 
 1   Open          6846 non-null   float64
 2   High          6846 non-null   float64
 3   Low           6846 non-null   float64
 4   Close         6846 non-null   float64
 5   Adj Close     6846 non-null   float64
 6   Volume        6846 non-null   int64  
 7   Dividends     6846 non-null   float64
 8   Stock Splits  6846 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 481.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4580 entries, 0 to 4579
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4580 non-null   int64  
 1   headline    4580 non-null   object 
 2   url         4580 non-null   object 
 3   publisher   4580 non-null   object 
 4   

In [29]:
stock_df_head = stock_df.head()
news_df_head = news_df.head()

stock_df_info, stock_df_head, news_df_info, news_df_head

(None,
          Date      Open      High       Low     Close  Adj Close      Volume  \
 0  1997-05-15  0.121875  0.125000  0.096354  0.097917   0.097917  1443120000   
 1  1997-05-16  0.098438  0.098958  0.085417  0.086458   0.086458   294000000   
 2  1997-05-19  0.088021  0.088542  0.081250  0.085417   0.085417   122136000   
 3  1997-05-20  0.086458  0.087500  0.081771  0.081771   0.081771   109344000   
 4  1997-05-21  0.081771  0.082292  0.068750  0.071354   0.071354   377064000   
 
    Dividends  Stock Splits  
 0        0.0           0.0  
 1        0.0           0.0  
 2        0.0           0.0  
 3        0.0           0.0  
 4        0.0           0.0  ,
 None,
    Unnamed: 0                                           headline  \
 0           0            Stocks That Hit 52-Week Highs On Friday   
 1           1         Stocks That Hit 52-Week Highs On Wednesday   
 2           2                      71 Biggest Movers From Friday   
 3           3       46 Stocks Moving In 

In [ ]:
# Step 1: Normalize Dates
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce').dt.date
stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce').dt.date

# Step 2: Perform Sentiment Analysis using TextBlob
def get_sentiment(text):
    return TextBlob(str(text)).sentiment.polarity

news_df['sentiment'] = news_df['headline'].apply(get_sentiment)


# Step 3: Aggregate Sentiment by Date
daily_sentiment = news_df.groupby('date')['sentiment'].mean().reset_index()

# Step 4: Compute Daily Stock Returns
stock_df = stock_df.sort_values(by='Date')
stock_df['return'] = stock_df['Close'].pct_change()


# Step 5: Merge Datasets on Date
merged_df = pd.merge(stock_df, daily_sentiment, how='inner', left_on='Date', right_on='date')

# Step 6: Calculate Pearson Correlation
correlation = merged_df[['return', 'sentiment']].corr().iloc[0, 1]

# Display merged dataset head and correlation value
merged_df_head = merged_df[['Date', 'Close', 'return', 'sentiment']].head()
correlation, merged_df_head